In [2]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import psycopg2

## Extract CSVs into DataFrames

In [3]:
census_df = pd.read_csv("census_data_county.csv", encoding="utf-8")
census_df.head()

,state,county,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Unemployment Count,Race: White,Race: African American,Race: Hispanic or Latino,"US Citizen, born in US","US Citizen, by naturalization"
0,Alabama,Autauga County,55200,37.8,58786.0,29372.0,8422.0,1065.0,42437,10565,1528,53385,579
1,Alabama,Baldwin County,208107,42.8,55962.0,31203.0,21653.0,4343.0,179526,19764,9353,198875,3319
2,Alabama,Barbour County,25782,39.9,34186.0,18461.0,6597.0,918.0,12216,12266,1106,25064,280
3,Alabama,Bibb County,22527,39.9,45340.0,20199.0,2863.0,658.0,17268,5018,547,22083,129
4,Alabama,Blount County,57645,40.8,48695.0,22656.0,8220.0,909.0,55054,862,5261,54877,876


In [4]:
votes_df = pd.read_csv("Resources/president_county_candidate.csv", encoding="utf-8")
votes_df.head()

,state,county,candidate,party,votes
0,Alabama,Autauga County,Donald Trump,REP,19764
1,Alabama,Autauga County,Joe Biden,DEM,7450
2,Alabama,Autauga County,Jo Jorgensen,LIB,346
3,Alabama,Autauga County,Write-ins,WRI,79
4,Alabama,Baldwin County,Donald Trump,REP,83055


In [5]:
biden_df = pd.read_csv("Resources/biden_tweets.csv", encoding="utf-8")
biden_df.head()

,tweet,city,country,state
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,NaN,USA,Florida
1,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,NaN,USA,Montana
2,#censorship #HunterBiden #Biden #BidenEmails #...,NaN,USA,California
3,"In 2020, #NYPost is being #censorship #CENSORE...",Chicago,USA,Illinois
4,"FBI Allegedly Obtained Hunter Biden Computer, ...",NaN,USA,Kentucky


In [6]:
trump_df = pd.read_csv("Resources/trump_tweets.csv", encoding="utf-8")
trump_df.head()

,tweet,city,country,state
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,NaN,USA,Florida
1,"#Trump: As a student I used to hear for years,...",Portland,USA,Oregon
2,You get a tie! And you get a tie! #Trump ‘s ra...,Washington,USA,District of Columbia
3,@CLady62 Her 15 minutes were over long time ag...,NaN,USA,California
4,@DeeviousDenise @realDonaldTrump @nypost There...,NaN,USA,Ohio


### Transform Census DataFrame

In [7]:
# Rename the column headers
census_transformed = census_df.rename(columns={"state": "State",
                                              "county": "County",
                                               "Population": "Population",
                                               "Median Age": "Median_Age",
                                               "Household Income": "Median_Household_Income",
                                               "Per Capita Income": "Per_Capita_Income",
                                               "Poverty Count": "Poverty_Count",
                                               "Unemployment Count": "Unemployment_Count",
                                               "Race: White": "Population_White",
                                               "Race: African American": "Population_AfricanAmerican",
                                               "Race: Hispanic or Latino": "Population_Hispanic",
                                               "US Citizen, born in US": "US_Citizen_Birth",
                                               "US Citizen, by naturalization": "US_Citizen_Naturalised"
                                              })

census_transformed = census_transformed[census_transformed['Median_Household_Income'].notna()]
census_transformed.shape

(3045, 13)

### Transform Votes Dataframe

In [8]:
# Rename columns
votes_transformed = votes_df.rename(columns={"state": "State",
                                        "county": "County",
                                         "candidate": "Candidate",
                                         "party": "Party",
                                         "votes": "Votes"
                                        })
votes_transformed

,State,County,Candidate,Party,Votes
0,Alabama,Autauga County,Donald Trump,REP,19764
1,Alabama,Autauga County,Joe Biden,DEM,7450
2,Alabama,Autauga County,Jo Jorgensen,LIB,346
3,Alabama,Autauga County,Write-ins,WRI,79
4,Alabama,Baldwin County,Donald Trump,REP,83055
...,...,...,...,...,...
19439,Wyoming,Weston County,Donald Trump,REP,3107
19440,Wyoming,Weston County,Joe Biden,DEM,360
19441,Wyoming,Weston County,Jo Jorgensen,LIB,46
19442,Wyoming,Weston County,Brock Pierce,IND,20


### Transform Biden Dataframe

In [9]:
# Create a filtered dataframe from specific columns
biden_cols = ["tweet", "country", "state"]
biden_transformed = biden_df[biden_cols].copy()

# Rename the column headers
biden_transformed = biden_transformed.rename(columns={"tweet": "Tweet",
                                                     "country": "Country",
                                                      "state": "State"
                                                     })

biden_transformed.index.name = 'ID'

biden_transformed.head()

# remove null state
biden_transformed = biden_transformed[biden_transformed['State'].notna()]
# biden_transformed[~biden_transformed['State'].isnull()]

In [28]:
biden_transformed.shape

(153579, 3)

### Transform Trump Dataframe

In [11]:
# Create a filtered dataframe from specific columns
trump_cols = ["tweet", "country", "state"]
trump_transformed = trump_df[trump_cols].copy()

# Rename the column headers
trump_transformed = trump_transformed.rename(columns={"tweet": "Tweet",
                                                     "country": "Country",
                                                      "state": "State"
                                                     })
# remove null state
trump_transformed = trump_transformed[trump_transformed['State'].notna()]

trump_transformed.index.name = 'ID'

trump_transformed.shape

(178885, 3)

### Create database connection

In [12]:
username = 'postgres'
password = 'postgres'
host = '35.197.164.165'
port = '5432'
database = 'postgres'

db_url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    username, password, host, port, database)

engine = sqlalchemy.create_engine(db_url)

conn = engine.connect()

In [13]:
# Confirm tables
engine.table_names()

['census_data', 'vote_data', 'biden_tweets', 'trump_tweets']

### Load DataFrames into Database (Grace's part)

In [158]:
# replace space in county column in census table
census_transformed["State"] = census_transformed["State"].str.replace(' ', '')

In [161]:
#load census data
census_transformed.to_sql(name='census_data', con=engine, if_exists='append', index=False)

#load vote data
votes_transformed.to_sql(name='vote_data', con=engine, if_exists='append', index=False)

#load biden tweet data
biden_transformed.to_sql(name='biden_tweets', con=engine, if_exists='append', index=True)

#load trump tweet data
trump_transformed.to_sql(name='trump_tweets', con=engine, if_exists='append', index=True)

In [349]:
# query = '''
#     select "State", "County", "Candidate", "Median_Household_Income", "Per_Capita_Income",
#     ("Votes"/"Population"::float) AS vote_percent 
#     from (
#         SELECT vote_data."State", vote_data."County", vote_data."Candidate", census_data."Population", 
#             vote_data."Votes", census_data."Median_Household_Income", census_data."Per_Capita_Income"
#             from vote_data 
#             JOIN census_data
#             ON census_data."County" = vote_data."County"
#             AND census_data."State" = vote_data."State") a
            
#     order by "Median_Household_Income" DESC
#     LIMIT 50
# '''

# pd.read_sql(query, engine)

In [386]:
# #Join vote data table and census data table and filter Trump
# vote_census = '''
# SELECT a."State",a."County",a."Candidate",a."Party",
#         b."Population",b."Median_Household_Income",b."Per_Capita_Income"
# from vote_data a
#     JOIN census_data b
#         ON a."County" = b."County"
#         AND a."State" = b."State"
# where a."Candidate" = 'Donald Trump'        
# order by"Median_Household_Income" desc
# '''
# pd.read_sql(vote_census, engine)

In [387]:
# #query Biden top 10 Votes county in join table - census & vote and filter Biden

# Biden_vote_census = '''

#     select *, ("Votes"/"Population"::float) AS vote_percent from (SELECT * from vote_data
#         JOIN census_data
#         ON census_data."County" = vote_data."County"
#         AND census_data."State" = vote_data."State") a
#     where a."Candidate" = 'Joe Biden'
    
#     order by "Median_Household_Income" DESC
#     LIMIT 10
    
# '''

# Biden_vote_census = pd.read_sql(Biden_vote_census, engine)
# Biden_vote_census

In [388]:
# #query Trump top 10 Votes county in join table - census & vote

# Trump_vote_census = '''
#     select a.*, ("Votes"/"Population"::float) AS vote_percent from (SELECT * from vote_data
#             JOIN census_data
#             ON census_data."County" = vote_data."County"
#             AND census_data."State" = vote_data."State") a
#         where a."Candidate" = 'Donald Trump'
    
#         order by vote_percent DESC
#         LIMIT 10
# '''

# Trump_vote_census = pd.read_sql(Trump_vote_census, engine)

# Trump_vote_census

### List the 5 state where Trump has his greatest leads over B.

In [437]:
query = '''
    select m."State", 
        ROUND("biden_vote_percent"::numeric, 4) AS biden_vote_percent, 
        ROUND("trump_vote_percent"::numeric, 4) AS trump_vote_percent, 
        ROUND(m."Median_Household_Income"::numeric, 2) AS Median_Household_Income, 
        m."Population", 
        ROUND(m."Per_Capita_Income"::numeric, 2) AS "Per_Capita_Income",
        ROUND(("trump_vote_percent"-"biden_vote_percent")::numeric, 4) AS "trump_leading"
    from (
        select 
            "State", 
            "Candidate" AS "biden_candidate", 
            AVG("Median_Household_Income") AS "Median_Household_Income", 
            AVG("Per_Capita_Income") AS "Per_Capita_Income",
            SUM("Votes") AS "Votes", 
            SUM("Population") AS "Population",
            (
            SUM("Votes")/SUM("Population")::float) AS biden_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Joe Biden'
        GROUP BY "State", "Candidate"
    ) m
    JOIN 
        (
            select 
            "State", 
            "Candidate" AS "trump_candidate", 
            (SUM("Votes")/SUM("Population")::float) AS trump_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Donald Trump'
        GROUP BY "State", "Candidate"
    ) n
    ON m."State" = n."State" 

    ORDER BY trump_leading DESC
    LIMIT 5
'''

results = pd.read_sql(query, engine)
results

,State,biden_vote_percent,trump_vote_percent,median_household_income,Population,Per_Capita_Income,trump_leading
0,Wyoming,0.1262,0.3325,59880.87,581836,31455.09,0.2063
1,Idaho,0.1700,0.3282,48432.39,1687809,24397.14,0.1582
2,Oklahoma,0.1286,0.2604,47482.55,3918137,24470.79,0.1318
3,Kentucky,0.1739,0.2987,43063.58,4440204,23117.96,0.1248
4,Alabama,0.1734,0.2948,41828.01,4864680,23072.84,0.1214


### List the 5 states where Biden has his greatest leads over Trump.

In [461]:
query = '''
    select m."State", 
        ROUND("biden_vote_percent"::numeric, 4) AS biden_vote_percent, 
        ROUND("trump_vote_percent"::numeric, 4) AS trump_vote_percent, 
        ROUND(m."Median_Household_Income"::numeric, 2) AS Median_Household_Income, 
        m."Population", 
        ROUND(m."Per_Capita_Income"::numeric, 2) AS "Per_Capita_Income",
        ROUND(("biden_vote_percent"-"trump_vote_percent")::numeric, 4) AS "biden_leading"
    from (
        select 
            "State", 
            "Candidate" AS "biden_candidate", 
            AVG("Median_Household_Income") AS "Median_Household_Income", 
            AVG("Per_Capita_Income") AS "Per_Capita_Income",
            SUM("Votes") AS "Votes", 
            SUM("Population") AS "Population",
            (
            SUM("Votes")/SUM("Population")::float) AS biden_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Joe Biden'
        GROUP BY "State", "Candidate"
    ) m
    JOIN 
        (
            select 
            "State", 
            "Candidate" AS "trump_candidate", 
            (SUM("Votes")/SUM("Population")::float) AS trump_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Donald Trump'
        GROUP BY "State", "Candidate"
    ) n
    ON m."State" = n."State"
    ORDER BY biden_leading DESC
    LIMIT 5
'''

results = pd.read_sql(query, engine)
results

,State,biden_vote_percent,trump_vote_percent,median_household_income,Population,Per_Capita_Income,biden_leading
0,Maryland,0.2858,0.1499,74890.71,6003435,36541.67,0.1358
1,California,0.2624,0.1372,64378.55,39148760,32089.62,0.1252
2,Hawaii,0.2573,0.1383,74450.50,1421954,32480.25,0.1190
3,Washington,0.3211,0.2132,56904.38,7294336,29278.85,0.1078
4,Delaware,0.3120,0.2113,63541.33,949495,32643.33,0.1008


### Query for top 5 states voe Kanye West - Order by Votes

In [445]:
query = '''
    select 
        "State", 
        "Candidate", 
        ROUND(AVG("Median_Household_Income")::numeric, 2) AS "Median_Household_Income", 
        AVG("Per_Capita_Income") AS "Per_Capita_Income",
        SUM("Votes") AS "Votes", 
        SUM("Population") AS "Population",
        (SUM("Votes")/SUM("Population")::float) AS vote_percent
    from (
        SELECT 
            vote_data."State", 
            vote_data."County", 
            vote_data."Candidate", 
            vote_data."Votes", 
            census_data."Median_Household_Income", 
            census_data."Population", 
            census_data."Per_Capita_Income"
        from vote_data 
            JOIN census_data
            ON census_data."County" = vote_data."County"
            AND census_data."State" = vote_data."State"
    ) a
    WHERE "Candidate" IN ('Kanye West')
    GROUP BY "State", "Candidate"
    ORDER BY "Votes" DESC
    LIMIT 5
'''

results = pd.read_sql(query, engine)
results

,State,Candidate,Median_Household_Income,Per_Capita_Income,Votes,Population,vote_percent
0,Tennessee,Kanye West,44977.20,24283.242105,10258,6651089,0.001542
1,Minnesota,Kanye West,59563.94,30827.471264,7946,5527358,0.001438
2,Colorado,Kanye West,56450.03,30695.406250,7862,5531141,0.001421
3,Utah,Kanye West,60981.86,25467.793103,6564,3045350,0.002155
4,Kentucky,Kanye West,43063.58,23117.958333,6556,4440204,0.001477


### For the states with the 10 highest household income, list them in order of their votes for Biden over Trump/Trump over Biden.

In [435]:
query = '''
    select m."State", 
        ROUND("biden_vote_percent"::numeric, 4) AS biden_vote_percent, 
        ROUND("trump_vote_percent"::numeric, 4) AS trump_vote_percent, 
        ROUND(m."Median_Household_Income"::numeric, 2) AS Median_Household_Income, 
        m."Population", 
        m."Per_Capita_Income",
        ROUND(("biden_vote_percent"-"trump_vote_percent")::numeric, 4) AS "biden_leading"
    from (
        select 
            "State", 
            "Candidate" AS "biden_candidate", 
            AVG("Median_Household_Income") AS "Median_Household_Income", 
            AVG("Per_Capita_Income") AS "Per_Capita_Income",
            SUM("Votes") AS "Votes", 
            SUM("Population") AS "Population",
            (
            SUM("Votes")/SUM("Population")::float) AS biden_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Joe Biden'
        GROUP BY "State", "Candidate"
    ) m
    JOIN 
        (
            select 
            "State", 
            "Candidate" AS "trump_candidate", 
            (SUM("Votes")/SUM("Population")::float) AS trump_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Donald Trump'
        GROUP BY "State", "Candidate"
    ) n
    ON m."State" = n."State" 
    ORDER BY "Median_Household_Income" DESC, "biden_leading" DESC
    LIMIT 10
'''

results = pd.read_sql(query, engine)
results

,State,biden_vote_percent,trump_vote_percent,median_household_income,Population,Per_Capita_Income,biden_leading
0,Maryland,0.2858,0.1499,74890.71,6003435,36541.666667,0.14
1,Hawaii,0.2573,0.1383,74450.50,1421954,32480.250000,0.12
2,California,0.2624,0.1372,64378.55,39148760,32089.620690,0.13
3,Delaware,0.3120,0.2113,63541.33,949495,32643.333333,0.10
4,Utah,0.1733,0.2689,60981.86,3045350,25467.793103,-0.10
5,Wyoming,0.1262,0.3325,59880.87,581836,31455.086957,-0.21
6,Nevada,0.2297,0.2172,59864.82,2922849,29034.176471,0.01
7,Minnesota,0.3109,0.2687,59563.94,5527358,30827.471264,0.04
8,Virginia,0.2857,0.2339,58501.29,8413774,30125.812030,0.05
9,Washington,0.3211,0.2132,56904.38,7294336,29278.846154,0.11


### Compare number of tweets from states where Biden has more vote than Trump

In [509]:
query = '''
    SELECT 
        a."State", 
        ROUND("biden_vote_percent"::numeric, 4) AS biden_vote_percent, 
        ROUND("trump_vote_percent"::numeric, 4) AS trump_vote_percent, 
        ROUND(a."median_household_income"::numeric, 2) AS median_household_income, 
        ROUND(("biden_vote_percent"-"trump_vote_percent")::numeric, 4) AS "biden_leading",
        b."trump_tweets",
        b."biden_tweets"
    FROM (
        select m."State", 
            ROUND("biden_vote_percent"::numeric, 4) AS biden_vote_percent, 
            ROUND("trump_vote_percent"::numeric, 4) AS trump_vote_percent, 
            ROUND(m."Median_Household_Income"::numeric, 2) AS Median_Household_Income, 
            m."Population", 
            m."Per_Capita_Income",
            ROUND(("biden_vote_percent"-"trump_vote_percent")::numeric, 4) AS "biden_leading"
        from (
            select 
                "State", 
                "Candidate" AS "biden_candidate", 
                AVG("Median_Household_Income") AS "Median_Household_Income", 
                AVG("Per_Capita_Income") AS "Per_Capita_Income",
                SUM("Votes") AS "Votes", 
                SUM("Population") AS "Population",
                (
                SUM("Votes")/SUM("Population")::float) AS biden_vote_percent
            from (
                SELECT 
                    vote_data."State", 
                    vote_data."County", 
                    vote_data."Candidate", 
                    vote_data."Votes", 
                    census_data."Median_Household_Income", 
                    census_data."Population", 
                    census_data."Per_Capita_Income"
                from vote_data 
                    JOIN census_data
                    ON census_data."County" = vote_data."County"
                    AND census_data."State" = vote_data."State"
            ) a
            WHERE "Candidate" = 'Joe Biden'
            GROUP BY "State", "Candidate"
    ) m
    JOIN 
        (
            select 
            "State", 
            "Candidate" AS "trump_candidate", 
            (SUM("Votes")/SUM("Population")::float) AS trump_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Donald Trump'
        GROUP BY "State", "Candidate"
    ) n
    ON m."State" = n."State" 
    ) a
    JOIN (
        SELECT a."State", "trump_tweets", "biden_tweets" FROM (
            SELECT "State", COUNT("State") AS "trump_tweets" FROM trump_tweets
            GROUP BY "State"
        ) a
        JOIN (
            SELECT "State", COUNT("State") AS "biden_tweets" FROM biden_tweets
            GROUP BY "State"
        ) b
        ON a."State" = b."State"
    ) b
    ON a."State" = b."State"
    ORDER BY "biden_leading" DESC
    LIMIT 10
'''

results = pd.read_sql(query, engine)
results

,State,biden_vote_percent,trump_vote_percent,median_household_income,biden_leading,trump_tweets,biden_tweets
0,Maryland,0.2858,0.1499,74890.71,0.1359,2189,2069
1,California,0.2624,0.1372,64378.55,0.1252,31148,25816
2,Hawaii,0.2573,0.1383,74450.50,0.1190,706,635
3,Washington,0.3211,0.2132,56904.38,0.1079,2759,2112
4,Delaware,0.3120,0.2113,63541.33,0.1007,245,331
5,Oregon,0.3252,0.2321,51121.89,0.0931,3126,2250
6,Colorado,0.3218,0.2434,56450.03,0.0784,3618,2687
7,Illinois,0.2381,0.1826,54839.62,0.0555,6777,5500
8,Virginia,0.2857,0.2339,58501.29,0.0518,2489,2226
9,Minnesota,0.3109,0.2687,59563.94,0.0422,1821,1622


### Compare number of tweets from states where Trump has more vote than Biden

In [514]:
query = '''
    SELECT 
        a."State", 
        ROUND("biden_vote_percent"::numeric, 4) AS biden_vote_percent, 
        ROUND("trump_vote_percent"::numeric, 4) AS trump_vote_percent, 
        ROUND(a."median_household_income"::numeric, 2) AS median_household_income, 
        ROUND(("biden_vote_percent"-"trump_vote_percent")::numeric, 4) AS "biden_leading",
        b."trump_tweets",
        b."biden_tweets"
    FROM (
        select m."State", 
            ROUND("biden_vote_percent"::numeric, 4) AS biden_vote_percent, 
            ROUND("trump_vote_percent"::numeric, 4) AS trump_vote_percent, 
            ROUND(m."Median_Household_Income"::numeric, 2) AS Median_Household_Income, 
            m."Population", 
            m."Per_Capita_Income",
            ROUND((-"biden_vote_percent"+"trump_vote_percent")::numeric, 4) AS "trump_leading"
        from (
            select 
                "State", 
                "Candidate" AS "biden_candidate", 
                AVG("Median_Household_Income") AS "Median_Household_Income", 
                AVG("Per_Capita_Income") AS "Per_Capita_Income",
                SUM("Votes") AS "Votes", 
                SUM("Population") AS "Population",
                (
                SUM("Votes")/SUM("Population")::float) AS biden_vote_percent
            from (
                SELECT 
                    vote_data."State", 
                    vote_data."County", 
                    vote_data."Candidate", 
                    vote_data."Votes", 
                    census_data."Median_Household_Income", 
                    census_data."Population", 
                    census_data."Per_Capita_Income"
                from vote_data 
                    JOIN census_data
                    ON census_data."County" = vote_data."County"
                    AND census_data."State" = vote_data."State"
            ) a
            WHERE "Candidate" = 'Joe Biden'
            GROUP BY "State", "Candidate"
    ) m
    JOIN 
        (
            select 
            "State", 
            "Candidate" AS "trump_candidate", 
            (SUM("Votes")/SUM("Population")::float) AS trump_vote_percent
        from (
            SELECT 
                vote_data."State", 
                vote_data."County", 
                vote_data."Candidate", 
                vote_data."Votes", 
                census_data."Median_Household_Income", 
                census_data."Population", 
                census_data."Per_Capita_Income"
            from vote_data 
                JOIN census_data
                ON census_data."County" = vote_data."County"
                AND census_data."State" = vote_data."State"
        ) a
        WHERE "Candidate" = 'Donald Trump'
        GROUP BY "State", "Candidate"
    ) n
    ON m."State" = n."State" 
    ) a
    JOIN (
        SELECT a."State", "trump_tweets", "biden_tweets" FROM (
            SELECT "State", COUNT("State") AS "trump_tweets" FROM trump_tweets
            GROUP BY "State"
        ) a
        JOIN (
            SELECT "State", COUNT("State") AS "biden_tweets" FROM biden_tweets
            GROUP BY "State"
        ) b
        ON a."State" = b."State"
    ) b
    ON a."State" = b."State"
    ORDER BY "trump_leading" DESC
    LIMIT 10
'''

results = pd.read_sql(query, engine)
results

,State,biden_vote_percent,trump_vote_percent,median_household_income,biden_leading,trump_tweets,biden_tweets
0,Wyoming,0.1262,0.3325,59880.87,-0.2063,269,238
1,Idaho,0.1700,0.3282,48432.39,-0.1582,608,404
2,Oklahoma,0.1286,0.2604,47482.55,-0.1318,847,806
3,Kentucky,0.1739,0.2987,43063.58,-0.1248,1484,1196
4,Alabama,0.1734,0.2948,41828.01,-0.1214,849,864
5,Arkansas,0.1398,0.2531,40786.80,-0.1133,613,469
6,Tennessee,0.1713,0.2781,44977.20,-0.1068,2253,1993
7,Nebraska,0.1937,0.2895,53038.53,-0.0958,394,374
8,Utah,0.1733,0.2689,60981.86,-0.0956,917,813
9,Montana,0.2340,0.3281,49172.32,-0.0941,173,287
